# Crypto Market Regime Classification - EDA

데이터 탐색 및 피처 엔지니어링 분석

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# 한글 폰트 설정 (Windows)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 스타일 설정
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

%matplotlib inline

## 1. 데이터 로드

In [ ]:
# 데이터 경로
data_dir = Path('../data')

# 모든 parquet 파일 로드
data_files = list(data_dir.glob('*.parquet'))
print(f"발견된 데이터 파일: {len(data_files)}")
for f in data_files:
    print(f"  - {f.name}")

In [ ]:
# 데이터 로드
dfs = {}
for f in data_files:
    symbol = f.stem  # 파일명에서 확장자 제거
    df = pd.read_parquet(f)
    dfs[symbol] = df
    print(f"{symbol}: {len(df)} rows, {df.index.min()} ~ {df.index.max()}")

In [ ]:
# BTC 데이터를 메인으로 사용
btc = dfs.get('BTC', list(dfs.values())[0])
print(f"\nBTC 데이터 shape: {btc.shape}")
btc.head()

In [ ]:
# 컬럼 정보
btc.info()

In [ ]:
# 기초 통계
btc.describe()

## 2. 데이터 품질 확인

In [ ]:
# 결측치 확인
print("결측치 현황:")
print(btc.isnull().sum())
print(f"\n결측치 비율: {btc.isnull().sum().sum() / btc.size * 100:.2f}%")

In [ ]:
# 중복 행 확인
duplicates = btc.index.duplicated().sum()
print(f"중복 인덱스: {duplicates}개")

In [ ]:
# OHLC 데이터 무결성 확인
invalid_ohlc = (
    (btc['high'] < btc['low']) | 
    (btc['high'] < btc['open']) | 
    (btc['high'] < btc['close']) |
    (btc['low'] > btc['open']) | 
    (btc['low'] > btc['close'])
).sum()
print(f"비정상 OHLC 데이터: {invalid_ohlc}개")

## 3. 가격 분포 및 추세 분석

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 종가 추이
axes[0, 0].plot(btc.index, btc['close'])
axes[0, 0].set_title('BTC 종가 추이')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Price (KRW)')

# 일일 수익률 분포
returns = btc['close'].pct_change().dropna()
axes[0, 1].hist(returns, bins=50, edgecolor='black', alpha=0.7)
axes[0, 1].axvline(returns.mean(), color='red', linestyle='--', label=f'Mean: {returns.mean():.4f}')
axes[0, 1].axvline(0, color='black', linestyle='-', alpha=0.3)
axes[0, 1].set_title('일일 수익률 분포')
axes[0, 1].set_xlabel('Return')
axes[0, 1].legend()

# 거래량 추이
axes[1, 0].bar(btc.index, btc['volume'], width=1, alpha=0.7)
axes[1, 0].set_title('거래량 추이')
axes[1, 0].set_xlabel('Date')
axes[1, 0].set_ylabel('Volume')

# 로그 수익률 QQ Plot
from scipy import stats
stats.probplot(returns, dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('수익률 QQ Plot (정규분포 비교)')

plt.tight_layout()
plt.show()

In [ ]:
# 수익률 통계
print("=== 일일 수익률 통계 ===")
print(f"평균: {returns.mean():.4f}")
print(f"표준편차: {returns.std():.4f}")
print(f"왜도(Skewness): {returns.skew():.4f}")
print(f"첨도(Kurtosis): {returns.kurtosis():.4f}")
print(f"최소: {returns.min():.4f}")
print(f"최대: {returns.max():.4f}")

## 4. 변동성 분석

In [ ]:
# 롤링 변동성 계산
btc['returns'] = btc['close'].pct_change()
btc['volatility_20'] = btc['returns'].rolling(20).std()
btc['volatility_60'] = btc['returns'].rolling(60).std()

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# 가격과 변동성
ax1 = axes[0]
ax1.plot(btc.index, btc['close'], label='Close Price')
ax1.set_ylabel('Price')
ax1.legend(loc='upper left')
ax1.set_title('BTC 가격 및 변동성')

ax2 = axes[1]
ax2.plot(btc.index, btc['volatility_20'], label='20-day Volatility', alpha=0.8)
ax2.plot(btc.index, btc['volatility_60'], label='60-day Volatility', alpha=0.8)
ax2.axhline(btc['volatility_20'].quantile(0.8), color='red', linestyle='--', alpha=0.5, label='80th percentile')
ax2.set_ylabel('Volatility')
ax2.set_xlabel('Date')
ax2.legend()

plt.tight_layout()
plt.show()

## 5. 레짐 레이블링 분석

In [ ]:
from src.labeling import RegimeLabeler

# 3-class 레이블링
labeler = RegimeLabeler(n_classes=3, trend_threshold=0.02)
labels = labeler.label(btc)

print("=== 레짐 분포 (3-class) ===")
label_counts = labels.value_counts()
for label, count in label_counts.items():
    pct = count / len(labels) * 100
    print(f"{label}: {count} ({pct:.1f}%)")

In [ ]:
# 레짐 시각화
btc['regime'] = labels

fig, ax = plt.subplots(figsize=(14, 6))

colors = {'BULL_TREND': 'green', 'BEAR_TREND': 'red', 'SIDEWAYS': 'gray'}

for regime in colors:
    mask = btc['regime'] == regime
    ax.scatter(btc.index[mask], btc['close'][mask], 
               c=colors[regime], label=regime, alpha=0.5, s=10)

ax.set_title('BTC 가격 및 레짐 분류')
ax.set_xlabel('Date')
ax.set_ylabel('Price')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 레짐별 수익률 통계
print("=== 레짐별 수익률 통계 ===")
for regime in ['BULL_TREND', 'BEAR_TREND', 'SIDEWAYS']:
    mask = btc['regime'] == regime
    regime_returns = btc.loc[mask, 'returns'].dropna()
    if len(regime_returns) > 0:
        print(f"\n{regime}:")
        print(f"  평균 수익률: {regime_returns.mean():.4f}")
        print(f"  표준편차: {regime_returns.std():.4f}")
        print(f"  샤프비율 (연율화): {regime_returns.mean() / regime_returns.std() * np.sqrt(252):.2f}")

## 6. 피처 분석

In [ ]:
from src.features import FeatureExtractor

# 피처 추출
extractor = FeatureExtractor()
features = extractor.transform(btc, dropna=True)

print(f"추출된 피처 수: {len(features.columns)}")
print(f"피처 목록:")
for i, col in enumerate(features.columns, 1):
    print(f"  {i}. {col}")

In [ ]:
# 피처 통계
features.describe()

In [ ]:
# 피처 상관관계
fig, ax = plt.subplots(figsize=(16, 14))
corr = features.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=False, cmap='RdBu_r', center=0,
            square=True, linewidths=0.5, ax=ax)
ax.set_title('피처 상관관계 히트맵')
plt.tight_layout()
plt.show()

In [ ]:
# 높은 상관관계 피처 쌍 찾기
high_corr_pairs = []
for i in range(len(corr.columns)):
    for j in range(i+1, len(corr.columns)):
        if abs(corr.iloc[i, j]) > 0.8:
            high_corr_pairs.append((
                corr.columns[i], 
                corr.columns[j], 
                corr.iloc[i, j]
            ))

print(f"높은 상관관계 (|r| > 0.8) 피처 쌍: {len(high_corr_pairs)}개")
for f1, f2, r in sorted(high_corr_pairs, key=lambda x: -abs(x[2]))[:10]:
    print(f"  {f1} <-> {f2}: {r:.3f}")

In [ ]:
# 레짐과 피처의 관계
aligned_labels = labels.loc[features.index]
features_with_regime = features.copy()
features_with_regime['regime'] = aligned_labels

# 레짐별 피처 평균
regime_means = features_with_regime.groupby('regime').mean()
print("레짐별 피처 평균:")
regime_means.T

In [ ]:
# 주요 피처 분포 (레짐별)
key_features = ['return_20d', 'volatility', 'rsi', 'ma_alignment', 'trend_strength']
key_features = [f for f in key_features if f in features.columns]

fig, axes = plt.subplots(1, len(key_features), figsize=(4*len(key_features), 4))
if len(key_features) == 1:
    axes = [axes]

for ax, feat in zip(axes, key_features):
    for regime in ['BULL_TREND', 'BEAR_TREND', 'SIDEWAYS']:
        mask = features_with_regime['regime'] == regime
        data = features_with_regime.loc[mask, feat].dropna()
        ax.hist(data, bins=30, alpha=0.5, label=regime)
    ax.set_title(feat)
    ax.legend()

plt.tight_layout()
plt.show()

## 7. 피처 중요도 분석 (Random Forest 기반)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# 데이터 준비
X = features.copy()
y = aligned_labels

# 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Random Forest 학습
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_scaled, y)

# 피처 중요도
importance = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)

# 시각화
fig, ax = plt.subplots(figsize=(10, 8))
importance.head(20).plot(kind='barh', ax=ax)
ax.set_title('피처 중요도 Top 20')
ax.set_xlabel('Importance')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
print("=== 피처 중요도 Top 20 ===")
for i, (feat, imp) in enumerate(importance.head(20).items(), 1):
    print(f"{i:2d}. {feat}: {imp:.4f}")

## 8. 멀티 심볼 분석

In [ ]:
# 모든 심볼 비교
fig, axes = plt.subplots(len(dfs), 1, figsize=(14, 4*len(dfs)), sharex=True)
if len(dfs) == 1:
    axes = [axes]

for ax, (symbol, df) in zip(axes, dfs.items()):
    # 정규화된 가격
    normalized = df['close'] / df['close'].iloc[0] * 100
    ax.plot(df.index, normalized, label=symbol)
    ax.set_ylabel('Normalized Price')
    ax.set_title(f'{symbol} 정규화 가격 추이')
    ax.legend()

plt.xlabel('Date')
plt.tight_layout()
plt.show()

In [ ]:
# 심볼간 상관관계
returns_df = pd.DataFrame()
for symbol, df in dfs.items():
    returns_df[symbol] = df['close'].pct_change()

print("심볼간 수익률 상관관계:")
returns_df.corr()

## 9. 요약 및 결론

In [ ]:
print("="*60)
print("EDA 요약")
print("="*60)

print(f"\n1. 데이터 현황")
print(f"   - 심볼 수: {len(dfs)}")
for symbol, df in dfs.items():
    print(f"   - {symbol}: {len(df)} rows ({df.index.min().date()} ~ {df.index.max().date()})")

print(f"\n2. 레짐 분포 (3-class)")
for label, count in label_counts.items():
    print(f"   - {label}: {count} ({count/len(labels)*100:.1f}%)")

print(f"\n3. 피처 현황")
print(f"   - 총 피처 수: {len(features.columns)}")
print(f"   - 높은 상관관계 쌍: {len(high_corr_pairs)}개")

print(f"\n4. 주요 피처 (Top 10)")
for i, (feat, imp) in enumerate(importance.head(10).items(), 1):
    print(f"   {i:2d}. {feat}: {imp:.4f}")